In [1]:
import pandas as pd
from ipyleaflet import Map, Marker, Circle, WidgetControl, Popup,AwesomeIcon
from ipywidgets import IntSlider, Layout, HTML
from geopy.distance import geodesic
import geohash as gh
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pyspark
from textblob import TextBlob
import re
from pyspark.sql.functions import *
import os
import numpy as np
import sys
from google.cloud import storage
import time
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType, DoubleType
from nltk.sentiment import SentimentIntensityAnalyzer
os.environ['PYSPARK_PYTHON'] = sys.executable


pd.set_option('display.max_columns', None)

In [3]:
all_city = []
filepath = "gs://jaime-project/"

# Code for finding all city folder in the blob
def find_city(bucket_name):
    l_review = []
    l_listing = []
    client = storage.Client()
    for blob in client.list_blobs(bucket_name, prefix=''):
        if 'reviews.csv' in str(blob):
            l_review.append(str(blob).split('/')[0].split(',')[1].replace(' ', ''))
        if 'listings.csv' in str(blob):
            l_listing.append(str(blob).split('/')[0].split(',')[1].replace(' ', ''))
    l = [x for x in l_review if x in l_listing]
    return list(set(l))
all_city = find_city('jaime-project')
all_city    
    

['paris',
 'Berlin',
 'Bangkok',
 'Amsterdam',
 'Toronto',
 'Bristol',
 'Brussels',
 'Cambridge',
 'Rome',
 'Edinburgh',
 'Barcelona',
 'london']

In [4]:
%%time

# Measure the running time for ingesting for 3 cities
df_merged = None

# Code to preprocess and add the data about reviews of the city into dataset
def ingest_data(df, city):
    # dataset loading and checking
    df_listings = spark.read.format('csv') \
        .option("header", "true") \
        .option("multiLine", "true") \
        .option("escape", "\"") \
        .option("quote", "\"") \
        .load(filepath+city+'/'+'listings.csv')
    df_reviews = spark.read.format('csv').option("header", 'true')\
        .option("multiLine", "true") \
        .option("escape", "\"") \
        .option("quote", "\"") \
        .load(filepath+city+'/'+'reviews.csv')
    
    # Add city name to the start of listing_id column in df_listings_new
    df_listings = df_listings.withColumn("id", concat(lit(city), df_listings["id"]))

    # Add city name to the start of listing_id column in df_review_new
    df_reviews = df_reviews.withColumn("listing_id", concat(lit(city), df_reviews["listing_id"]))
        
    # Select specific columns from the DataFrame
    df_listings = df_listings.select("id", "price", "longitude", "latitude", "name", "listing_url", "review_scores_rating")
    df_reviews = df_reviews.select("listing_id", "reviewer_id", "comments")

    df_merged = df_reviews.join(df_listings, df_reviews.listing_id == df_listings.id, how='left')
    df_merged = df_merged.dropna(how='any')
    
    print('data for ', city)
    df_merged.show()
    
    if df is None:
        return df_merged
    else:
        return  df.union(df_merged)
    
ti = []
for i, c in enumerate(all_city):
    st = time.perf_counter()
    df_merged = ingest_data(df_merged, c)
    fi = time.perf_counter()
    ti.append(fi-st)

data for  paris


+----------+-----------+--------------------+---------+-------+---------+--------+--------------------+--------------------+--------------------+
|listing_id|reviewer_id|            comments|       id|  price|longitude|latitude|                name|         listing_url|review_scores_rating|
+----------+-----------+--------------------+---------+-------+---------+--------+--------------------+--------------------+--------------------+
| paris3109|   51636494|Tout s'est bien d...|paris3109|$150.00|   2.3187|48.83191|Rental unit in Pa...|https://www.airbn...|                 5.0|
| paris3109|    4142888|Un petit nid foui...|paris3109|$150.00|   2.3187|48.83191|Rental unit in Pa...|https://www.airbn...|                 5.0|
| paris3109|    7415343|Appartement spaci...|paris3109|$150.00|   2.3187|48.83191|Rental unit in Pa...|https://www.airbn...|                 5.0|
| paris3109|   21159216|Appartement total...|paris3109|$150.00|   2.3187|48.83191|Rental unit in Pa...|https://www.airbn...|

+------------+-----------+--------------------+------------+------+---------+--------+--------------------+--------------------+--------------------+
|  listing_id|reviewer_id|            comments|          id| price|longitude|latitude|                name|         listing_url|review_scores_rating|
+------------+-----------+--------------------+------------+------+---------+--------+--------------------+--------------------+--------------------+
|  Berlin3176|      21475|excellent stay, i...|  Berlin3176|$83.00|  13.4181|52.53471|Fabulous Flat in ...|https://www.airbn...|                4.63|
|Berlin265408|   15685924|Ich kann bestätig...|Berlin265408|$20.00| 13.44602|52.50977|Elegant 4-bed roo...|https://www.airbn...|                4.45|
|Berlin265408|   26757394|It's just a hoste...|Berlin265408|$20.00| 13.44602|52.50977|Elegant 4-bed roo...|https://www.airbn...|                4.45|
|  Berlin3176|     263467|Britta's apartmen...|  Berlin3176|$83.00|  13.4181|52.53471|Fabulous Flat 

+--------------+-----------+--------------------+--------------+---------+---------+--------+--------------------+--------------------+--------------------+
|    listing_id|reviewer_id|            comments|            id|    price|longitude|latitude|                name|         listing_url|review_scores_rating|
+--------------+-----------+--------------------+--------------+---------+---------+--------+--------------------+--------------------+--------------------+
|  Bangkok27934|    1368195|We stayed in the ...|  Bangkok27934|$1,903.00|100.54134|13.75983|Condo in Samsen N...|https://www.airbn...|                4.85|
|  Bangkok27934|    2007324|My girlfriend and...|  Bangkok27934|$1,903.00|100.54134|13.75983|Condo in Samsen N...|https://www.airbn...|                4.85|
|Bangkok1296322|    7280739|It was a real ple...|Bangkok1296322|  $700.00|100.51332|13.71425|Rental unit in Ba...|https://www.airbn...|                4.47|
|Bangkok1296322|    6309642|Great place and l...|Bangkok12

+-------------+-----------+--------------------+-------------+------+---------+--------+--------------------+--------------------+--------------------+
|   listing_id|reviewer_id|            comments|           id| price|longitude|latitude|                name|         listing_url|review_scores_rating|
+-------------+-----------+--------------------+-------------+------+---------+--------+--------------------+--------------------+--------------------+
|Amsterdam2818|      10952|Daniel is really ...|Amsterdam2818|$82.00|  4.94358|52.36435|Quiet Garden View...|https://www.airbn...|                 4.9|
|Amsterdam2818|      12798|Daniel is the mos...|Amsterdam2818|$82.00|  4.94358|52.36435|Quiet Garden View...|https://www.airbn...|                 4.9|
|Amsterdam2818|      11869|We had such a gre...|Amsterdam2818|$82.00|  4.94358|52.36435|Quiet Garden View...|https://www.airbn...|                 4.9|
|Amsterdam2818|      14064|Very professional...|Amsterdam2818|$82.00|  4.94358|52.36435|

...

data for  Rome


+----------+-----------+--------------------+--------+-------+---------+--------+--------------------+--------------------+--------------------+
|listing_id|reviewer_id|            comments|      id|  price|longitude|latitude|                name|         listing_url|review_scores_rating|
+----------+-----------+--------------------+--------+-------+---------+--------+--------------------+--------------------+--------------------+
|  Rome2737|   24333039|This spot was in ...|Rome2737| $50.00| 12.48215|41.87136|Place to stay in ...|https://www.airbn...|                 4.8|
|  Rome2737|   10493776|Elif was really v...|Rome2737| $50.00| 12.48215|41.87136|Place to stay in ...|https://www.airbn...|                 4.8|
|  Rome2737|   29630601|Elif's flat is sp...|Rome2737| $50.00| 12.48215|41.87136|Place to stay in ...|https://www.airbn...|                 4.8|
|  Rome2737|   31215535|I had a wonderful...|Rome2737| $50.00| 12.48215|41.87136|Place to stay in ...|https://www.airbn...|       

+--------------+-----------+--------------------+--------------+-------+---------+--------+--------------------+--------------------+--------------------+
|    listing_id|reviewer_id|            comments|            id|  price|longitude|latitude|                name|         listing_url|review_scores_rating|
+--------------+-----------+--------------------+--------------+-------+---------+--------+--------------------+--------------------+--------------------+
|Edinburgh15420|     186358|My wife and I sta...|Edinburgh15420|$120.00| -3.18805|55.95759|Georgian Boutique...|https://www.airbn...|                4.98|
|Edinburgh15420|      95218|Charlotte couldn'...|Edinburgh15420|$120.00| -3.18805|55.95759|Georgian Boutique...|https://www.airbn...|                4.98|
|Edinburgh15420|     429751|I went to Edinbur...|Edinburgh15420|$120.00| -3.18805|55.95759|Georgian Boutique...|https://www.airbn...|                4.98|
|Edinburgh15420|     420830|This flat was inc...|Edinburgh15420|$120.0

+--------------+-----------+--------------------+--------------+-------+---------+--------+--------------------+--------------------+--------------------+
|    listing_id|reviewer_id|            comments|            id|  price|longitude|latitude|                name|         listing_url|review_scores_rating|
+--------------+-----------+--------------------+--------------+-------+---------+--------+--------------------+--------------------+--------------------+
|Barcelona17475|     141807|Excellent host, b...|Barcelona17475|$165.00|  2.17044|41.39939|Attic Sagrada Fam...|https://www.airbn...|                4.48|
|Barcelona17475|      68687|Luca is very pers...|Barcelona17475|$165.00|  2.17044|41.39939|Attic Sagrada Fam...|https://www.airbn...|                4.48|
|Barcelona17475|      96055|I am glad we foun...|Barcelona17475|$165.00|  2.17044|41.39939|Attic Sagrada Fam...|https://www.airbn...|                4.48|
|Barcelona17475|     171084|I should start by...|Barcelona17475|$165.0

+-----------+-----------+--------------------+-----------+------+---------+--------+--------------------+--------------------+--------------------+
| listing_id|reviewer_id|            comments|         id| price|longitude|latitude|                name|         listing_url|review_scores_rating|
+-----------+-----------+--------------------+-----------+------+---------+--------+--------------------+--------------------+--------------------+
|london13913|     177109|My girlfriend and...|london13913|$70.00|  -0.1127|51.56861|Rental unit in Is...|https://www.airbn...|                 4.8|
|london13913|   19835707|Alina was a reall...|london13913|$70.00|  -0.1127|51.56861|Rental unit in Is...|https://www.airbn...|                 4.8|
|london13913|    1110304|Alina is an amazi...|london13913|$70.00|  -0.1127|51.56861|Rental unit in Is...|https://www.airbn...|                 4.8|
|london13913|    1216358|Alina's place is ...|london13913|$70.00|  -0.1127|51.56861|Rental unit in Is...|https:/

In [5]:
for i, c in enumerate(ti):
    print("time to ingest", i, "th cities ", c," seconds")
    

time to ingest 0 th cities  23.364097638000203  seconds
time to ingest 1 th cities  5.633049764999669  seconds
time to ingest 2 th cities  5.841732721999506  seconds
time to ingest 3 th cities  4.492692631999489  seconds
time to ingest 4 th cities  4.963206518000334  seconds
time to ingest 5 th cities  3.9122320160004165  seconds
time to ingest 6 th cities  4.05107863900048  seconds
time to ingest 7 th cities  4.040764360000139  seconds
time to ingest 8 th cities  4.744367688999773  seconds
time to ingest 9 th cities  3.546610198000053  seconds
time to ingest 10 th cities  4.354166850000183  seconds
time to ingest 11 th cities  5.752213177000158  seconds


Time to ingest data between using Spark and Pandas DataFrame is quite similar, with around 6 seconds.

In [6]:
df_merged.show()

+----------+-----------+--------------------+---------+-------+---------+--------+--------------------+--------------------+--------------------+
|listing_id|reviewer_id|            comments|       id|  price|longitude|latitude|                name|         listing_url|review_scores_rating|
+----------+-----------+--------------------+---------+-------+---------+--------+--------------------+--------------------+--------------------+
| paris3109|   51636494|Tout s'est bien d...|paris3109|$150.00|   2.3187|48.83191|Rental unit in Pa...|https://www.airbn...|                 5.0|
| paris3109|    4142888|Un petit nid foui...|paris3109|$150.00|   2.3187|48.83191|Rental unit in Pa...|https://www.airbn...|                 5.0|
| paris3109|    7415343|Appartement spaci...|paris3109|$150.00|   2.3187|48.83191|Rental unit in Pa...|https://www.airbn...|                 5.0|
| paris3109|   21159216|Appartement total...|paris3109|$150.00|   2.3187|48.83191|Rental unit in Pa...|https://www.airbn...|

In [7]:
# Download the VADER lexicon
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [8]:
print('Size of cur dataframe is ', df_merged.count())

Size of cur dataframe is  8058086


In [9]:
%%time

# Initialize the SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Define a UDF to compute sentiment scores
def sentiment_score_nltk(text):
    if text:
        return sia.polarity_scores(text)['compound']
    else:
        return None

sentiment_nltk_udf = udf(sentiment_score_nltk, FloatType())

# Apply the UDF to create a sentiment score column
df_merged = df_merged.withColumn("sentiment_score", sentiment_nltk_udf(df_merged["comments"]))

def convert_price(price):
    price = price.replace(',','')
    if price and len(price) > 0 and not price[0].isdigit():
        return float(price[1:])
    try:
        return float(price)
    except:
        return None

# Convert price to numerical character
convert_price_udf = udf(convert_price, DoubleType())
df_merged = df_merged.withColumn("price", convert_price_udf(col("price")))

# Filter out empty or null comments
df_merged = df_merged.filter(col("comments").isNotNull() & (col("comments") != ""))

# Remove entries with the cancellation comment
df_merged = df_merged.filter(~col("comments").like("Host canceled this booking%"))

# Text normalization
def normalize_text(text):
    # Remove numerical characters
    text = re.sub(r'\d+', '', text)
    return text

normalize_text_udf = udf(normalize_text)
df_merged = df_merged.withColumn("comments", normalize_text_udf(col("comments")))

# Filtered null data
df_merged = df_merged.na.drop()

df_merged.show()


24/05/07 00:11:46 WARN DAGScheduler: Broadcasting large task binary with size 1042.0 KiB


+----------+-----------+--------------------+---------+-----+---------+--------+--------------------+--------------------+--------------------+---------------+
|listing_id|reviewer_id|            comments|       id|price|longitude|latitude|                name|         listing_url|review_scores_rating|sentiment_score|
+----------+-----------+--------------------+---------+-----+---------+--------+--------------------+--------------------+--------------------+---------------+
| paris3109|   51636494|Tout s'est bien d...|paris3109|150.0|   2.3187|48.83191|Rental unit in Pa...|https://www.airbn...|                 5.0|         -0.128|
| paris3109|    4142888|Un petit nid foui...|paris3109|150.0|   2.3187|48.83191|Rental unit in Pa...|https://www.airbn...|                 5.0|            0.0|
| paris3109|    7415343|Appartement spaci...|paris3109|150.0|   2.3187|48.83191|Rental unit in Pa...|https://www.airbn...|                 5.0|            0.0|
| paris3109|   21159216|Appartement tota

In [10]:
df_merged.show()

24/05/07 00:11:58 WARN DAGScheduler: Broadcasting large task binary with size 1042.0 KiB


+----------+-----------+--------------------+---------+-----+---------+--------+--------------------+--------------------+--------------------+---------------+
|listing_id|reviewer_id|            comments|       id|price|longitude|latitude|                name|         listing_url|review_scores_rating|sentiment_score|
+----------+-----------+--------------------+---------+-----+---------+--------+--------------------+--------------------+--------------------+---------------+
| paris3109|   51636494|Tout s'est bien d...|paris3109|150.0|   2.3187|48.83191|Rental unit in Pa...|https://www.airbn...|                 5.0|         -0.128|
| paris3109|    4142888|Un petit nid foui...|paris3109|150.0|   2.3187|48.83191|Rental unit in Pa...|https://www.airbn...|                 5.0|            0.0|
| paris3109|    7415343|Appartement spaci...|paris3109|150.0|   2.3187|48.83191|Rental unit in Pa...|https://www.airbn...|                 5.0|            0.0|
| paris3109|   21159216|Appartement tota

In [11]:
%%time

from pyspark.sql import functions as F

# Grouped the reviews and get the mean sentiment score for each listing
result = df_merged.groupBy("listing_id").agg(
    F.mean('price').alias('price'),
    F.mean('sentiment_score').alias('sentiment_score'),
    F.first('latitude').alias('latitude'),
    F.first('longitude').alias('longitude'),
    F.first('name').alias('name'),
    F.first('listing_url').alias('listing_url'),
    F.first('review_scores_rating').alias('review_scores_rating')
)

# To display the DataFrame
result.show()

24/05/07 00:12:12 WARN DAGScheduler: Broadcasting large task binary with size 1030.5 KiB


+--------------------+-----+--------------------+------------------+------------------+--------------------+--------------------+--------------------+
|          listing_id|price|     sentiment_score|          latitude|         longitude|                name|         listing_url|review_scores_rating|
+--------------------+-----+--------------------+------------------+------------------+--------------------+--------------------+--------------------+
|Amsterdam10004460...|200.0| 0.27250000536441804|        52.3682435|         4.8984594|Complete studio A...|https://www.airbn...|                 4.4|
|Amsterdam10006410...|225.0|  0.7418000102043152|52.390942060117304| 4.912909224236625|Cozy Longstay Apt...|https://www.airbn...|                 5.0|
|Amsterdam10012719...|340.0|  0.4921000003814697|        52.3605129|         4.8585385|Brand new apartme...|https://www.airbn...|                 5.0|
|Amsterdam10015204...|193.0| -0.9136000275611877|        52.3685844|         4.8944702|Lovely 

In [12]:
%%time

# Compute the squared error
result = result.withColumn("squared_error", ((col("review_scores_rating") / 5 - col("sentiment_score")) ** 2))
# Compute the mean of the squared errors
mean_squared_error = result.select(avg("squared_error").alias("mse")).first()["mse"]
rmse = np.sqrt(mean_squared_error)
print("RMSE is ", rmse)

RMSE is  0.45466609491978205
CPU times: user 574 ms, sys: 99.7 ms, total: 674 ms
Wall time: 54min 6s


In [13]:
%%time

# Write the DataFrame to CSV format in the specified bucket path
result.write.csv(filepath+'res.csv', header=True, mode="overwrite")

24/05/07 02:00:24 WARN DAGScheduler: Broadcasting large task binary with size 1030.5 KiB
24/05/07 02:54:16 WARN DAGScheduler: Broadcasting large task binary with size 1083.7 KiB


CPU times: user 559 ms, sys: 113 ms, total: 673 ms
Wall time: 54min 37s
